# CNN Image Annotation with Napari

### Welcome! 

This notebook allows you to import your raw microscopy data into [_Napari_](https://napari.org/ "Napari: a fast, interactive, multi-dimensional image viewer for Python"), a fast, interactive, multi-dimensional image viewer for Python. Follow the step-wise instructions to annotate your large, multi-dimensional images and extract single-cell image patches corresponding to different states of the cell cycle. 

This is a preview of the *napari* interface for image annotations:
![image](../assets/napari_annotator.png)


### Running Instructions:

1. To run this notebook, go to ```Kernel``` and click on ```Restart & Run All```. Alternatively, for a shortcut, click on the ```⏩ ``` in the top dashboard. It's the icon right next to the ```Run ▶️```, ``` ⏹️```, ``` 🔁``` and the ```Code``` dropdown window. This will restart the notebook & run all of its cells.

2. Calling the last cell will start a separate Python window with the *napari* widget. Please allow a few seconds to load the graphical user interface (GUI) to load. You should see your movie loaded on the screen, such as here:



### Important Notes:

- Make sure you have the latest version of *napari* installed in your virtual environment. If in doubt, run ```pip install napari -U``` for installing the upgrade. 
- Do not forget to export your annotation zip file before terminating the session. The annotations will not be autosaved nor exported automatically, so failing to export your annotations will result in your work being lost!
- Keep in mind that not all of your annotations have to be stored in a single zip file - you can create as many annotations as you'd like and spread out your annotated dataset across multiple zip files.

---

## Annotating the Single-Cell Image Patches:

To train a neural network to automatically and accurately classify previously unseen single-cell images, we need to manually annotate a handful of such image patches with a label. This is because the learning approach we use to train such a network is *supervised* (by a human annotator) - this means that for the learning algorithm to master the patterns representative for each class, we need to provide some of the images with the correct answer, often referred to as the *'ground truth'*. This allows the network to make high-fidelity predictions about previously unseen images.

In this section, a random sample of labeled images needs to be provided by annotating any microscopy movie of choice. The rule of thumb is that the more examples you can generate the better. We recommend to label at least **500 instances per each class** for reasonably accurate training. Using the guide below, please annotate each image with one of the five labels provided.

Here are some examples of cells and their corresponding labels:
![image](../assets/cell_cycle_states.png)


### Annotation Instructions:

1. Annotate your movie by clicking on the individual instances of the labels you wish to annotate. Although there is no sub-pixels targetting accuracy required at this stage, please aim to click at the centre of each cell / nucleus to allow the image patches to be cropped around those coordinates properly. The default labels is *interphase*.

2. Change the labels for which you wish to annotate at the bottom left corner of the GUI by choosing the appropriate label from the dropdown menu. There should be 5 labels available: *interphase (default), prometaphase, metaphase, anaphase and apoptosis*. You can swiftly change between the class labels by pressing ```.``` or ```,``` keyboard key for the next or previous label, respectively.

3. When done annotating one image, you can move to the next image by clicking on the ```right arrow``` or ```left arrow``` on the slider at the bottom of the GUI. Alternatively, you can select the image layer in the left panel and use the keyboard left/right keys or Ctrl-scroll to browse.

4. To erase a mislabelled point from the annotations list, choose the ```Delete point``` button at the top left corner of the GUI, or press the `backspace` key. This will only delete the latest point. You can delete many mislabelled points by clicking on the ```Multi-point selection``` button (or by pressing the `S`-key), holding `shift` as you select individual cells, then using the ```Delete point``` button (or `backspace` key). 

5. When done annotating, export the annotation file by clicking the ```Export``` button at the very bottom right corner of the GUI. **Do not forget to export your annotations before terminating the session. The annotations will not be autosaved nor exported automatically.** 

6. *Have you exported the annotations?* Close the GUI by clicking on the red ```X``` to quit the python GUI completely. *Optional:* If using OS X, choose to 'Force quit' the napari window for completeness. Doing so will cause this notebook's kernel to die. This outcome is fine as this notebook task is finished.

---

**Happy annotating!**

*Your [CellX](http://lowe.cs.ucl.ac.uk/cellx.html "Lowe Lab @ UCL") team*


### Import some useful libraries:

We first need to load some libraries of code that will help with the data processing and visualization.

In [1]:
import enum
import io
import os
import json
import napari
import numpy as np

from magicgui import magicgui
from skimage.io import imread
from skimage.io import imsave
from skimage.io import imshow

from pathlib import Path
from napari.layers import Image
from datetime import datetime
from zipfile import ZipFile


### Provide the directory for the movie you wish to annotate:

In this example, we provide an example 50-frame long crop (600 x 450 pixels) of a time-lapse microscopy movie ```MDCK_H2B_GFP_movie.tif``` of MDCK cells expressing an *H2B-GFP* fluorescent tag, which visualises the nuclei of the individual cells. This movie has dimensions (*i.e.* shape) of ```(50, 450, 600)```. When loading your own movie, make sure it is saved as a ```tif``` file and provide its absolute path.

Unit conversion for example data: *1 µm = 3 pixels, 1 frame = 4 minutes*


In [2]:
filename = "../data/MDCK_H2B_GFP_movie.tif"
print (filename)


../data/MDCK_H2B_GFP_movie.tif


### Load the image data from the movie:

In [3]:
data = imread(filename)
metadata = {"filename": filename}


### Define the cell cycle states which you'd like your CNN to categorise:

Create an object with enumerated classes which you want to classify. In this particular example, we provide 4 classes of actively dividing cells (*i.e. **interphase** pooled for G1-, S- and G2-phases & **pro(meta)phase, metaphase & ana(telo)phase** for specific phases of cell division*) and 1 class for ceasing cells (*i.e. **apoptosis***). 

To familiarise yourself with the typical cell morphologies & chromatin condensation of an actively dividing cell, please see the example images below:

![image](../assets/cell_cycle_states.png)


In [4]:
@enum.unique
class CellState(enum.Enum):
    Interphase = 0
    Prometaphase = 1
    Metaphase = 2
    Anaphase = 3
    Apoptosis = 4
    

### Assign the colours to individual labelled states:

The default settings will follow the ```matplotlib``` standard colour library with {"blue", "orange", "green", "red", "purple"}. 

*Note:* When changing the default setting or defining new colour palettes, please specify the [HEX code](https://www.color-hex.com/ "HEX Color Codes") for new colours.


In [5]:
COLOR_CYCLE = [
    '#1f77b4', # blue
    '#ff7f0e', # orange
    '#2ca02c', # green
    '#d62728', # red
    '#9467bd', # purple
]

### This function crops an image patch around the labelled points:

Default setting will crop a 64 x 64 pixel square image patch with the labelled point at the centre of the patch, i.e. 32 pixels up, 32 pixels down, 32 pixels left & 32 pixels right from the labelled coordinates. Please only alter with care.


In [6]:
def get_image_patch(layers, coords, shape=64):
    """ Get an image patch from the image layer data. """
    
    flagged = False
    square = shape // 2

    patches = []
    
    for layer in layers:
        
        # Read the image from layers:
        frame, y_coo, x_coo = [int(coo) for coo in coords]
        image = layer.data[frame]
        im_h, im_w = image.shape
        edge_dist = [y_coo, im_h - y_coo, x_coo, im_w - x_coo]
        
        # Check if padding is needed:
        if any([item < square for item in edge_dist]):
            pad_width = int(square - min(edge_dist))
            image = np.pad(array=image, pad_width=pad_width, mode='constant')
            y_coo = y_coo + pad_width
            x_coo = x_coo + pad_width
            flagged = True

        # Slice the patch from the image:
        patch = image[y_coo-square : y_coo+square, x_coo-square : x_coo+square].astype(np.uint8)
        patches.append(patch) 
        
    # Check if all patches are of specified shape:
    #if not all([patch.shape == (shape, shape) for patch in patches[0]]):
    #    raise ValueError(f"Image patches don't have correct shape: {[patch.shape for patch in patches[0]]}")
    
    return np.stack(patches, axis=-1), flagged
    

### The annotator function:

For those users more experienced at Python, you can read the code the below & possibly add more ```key_bindings``` functions for an even smoother annotation in *napari*: visit this [guide](https://napari.org/docs/0.3.8/_modules/napari/utils/key_bindings.html "Source code for napari.utils.key_bindings"). Otherwise, no manipulation is encouraged at this step.


In [7]:
def annotator(viewer):
    
    SESSION_TIME = datetime.now().strftime("%m-%d-%Y--%H-%M-%S")
    SESSION_NAME = f"annotation_{SESSION_TIME}"
    
    # add an empty points layer, with the same dimensions as the image data
    points_layer = viewer.add_points(
        name="Annotation", 
        properties={'State': [s.name for s in CellState]}, 
        ndim=data.ndim
    )

    points_layer.mode = 'add'
    points_layer.face_color = 'State'
    points_layer.face_color_cycle = COLOR_CYCLE
    points_layer.face_color_mode = 'cycle'

    
    @magicgui(
        call_button="Export",
        layout="horizontal",
        filename={"label": "Export path:"},
    )
    
    def cnn_annotation_widget(
        filename: Path = Path.home(),  # path objects are provided a file picker
        shape: int = 64,
        use_visible_layers: bool = True,
        state: CellState = list(CellState)[0],
    ):
        """ Export the annotations: """
        
        export_data = {'shape': shape}
        
        # find the visible image layers and export the metadata
        image_layers = [layer for layer in viewer.layers if isinstance(layer, Image)]
        for layer in image_layers:
            if use_visible_layers and layer.visible:
                export_data[layer.name] = layer.metadata
        
        # record the coordinates of the annotations 
        for idx in range(points_layer.data.shape[1]):
            export_data[f'coords-{idx}'] = points_layer.data[:, idx].tolist()
        
        # record the state labels of the annotations 
        export_data['labels'] = points_layer.properties['State'].tolist()
        
        # TODO: say whether patch is flagged or not:
        export_data['flagged'] = [False for _ in points_layer.properties['State']]
        
        # serialize the CellState labels
        export_data['states'] = {s.name: s.value for s in CellState}
        
        # extract the image patches here
        with ZipFile(f"../data/{SESSION_NAME}.zip", 'w') as myzip:
            for idx, patch_coords in enumerate(points_layer.data):
                
                patch_label = points_layer.properties['State'][idx]

                # grab the image patch
                image_patches, flagged = get_image_patch(image_layers, patch_coords, shape=shape)
                
                # check if the item was flagged:
                export_data['flagged'][idx] = flagged
                suffix = '_flagged' if flagged else ''
                image_patch_fn = f"{patch_label}/{patch_label}_{SESSION_TIME}_{idx}{suffix}.tif"
                
                # open a stream to write to the zip file
                stream = io.BytesIO()
                imsave(stream, image_patches, format='tif')
                stream_data = stream.getvalue()
                myzip.writestr(image_patch_fn, stream_data)
        
            # write out the json log to the zip file also
            stream = json.dumps(export_data, indent=2)
            myzip.writestr(f"{SESSION_NAME}.json", stream)
        
        print (f"JSON file & image patches have been exported.\n'../data/{SESSION_NAME}.zip'")
        
        return locals().values()
    
    def _change_points_properties(event):
        """ Update the current properties of the points layer to reflect the currently selected state. """
        points_layer.current_properties['State'] = np.array([cnn_annotation_widget.state.value.name])
    
    cnn_annotation_widget.state.changed.connect(_change_points_properties)
    
    # add the magicgui dock widget 
    viewer.window.add_dock_widget(cnn_annotation_widget)
    
    @viewer.bind_key('.')
    def next_label(event=None):
        """ Increment the label in the GUI """
        new_state = (cnn_annotation_widget.state.value.value + 1) % len(CellState)
        cnn_annotation_widget.state.value = CellState(new_state)
        
    @viewer.bind_key(',')
    def previous_label(event=None):
        """ Decrement the label in the GUI """
        new_state = (cnn_annotation_widget.state.value.value - 1) % len(CellState)
        cnn_annotation_widget.state.value = CellState(new_state)


### Running the cell below will open *napari* in a separate Python window. 

*Note:* ***Please allow a few seconds for the GUI to load.***

In [8]:
with napari.gui_qt():
    
    viewer = napari.Viewer()
    viewer.add_image(data, name='GFP', metadata=metadata)
   
    annotator(viewer)
    

JSON file & image patches have been exported.
'../data/annotation_03-16-2021--10-58-50.zip'


### Export the annotations:

1. Click on the ```Export``` button at the bottom right corner of the *napari* window when done annotating.
2. The name of the exported file will be printed under the cell above. Please check it was saved successfully in the folder.
3. Close the *napari* window & quit the python GUI completely.

(Doing so will cause this notebook's kernel to die. This outcome is fine as this notebook task is finished.)


### Next steps:

1. If you prefer to check the quality of your image patches and visualise some of your overall data statistics, please visit the **B_CNN_Data_Preview_Images.ipynb** iPython notebook
2. If you'd like to proceed directly to the neural network training step in the Google Colab environment, please visit the **C_CNN_Training_and_Validation.ipynb** iPython notebook


#### Done! You can close this notebook now.